In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# Load the dataset
url = "https://raw.githubusercontent.com/camilousa/datasets/refs/heads/master/healthcare-dataset-stroke-data.csv"
df = pd.read_csv(url)

In [4]:
# Exploración de datos
print("Forma del conjunto de datos:", df.shape)
print("\nInformación del conjunto de datos:")
df.info()

Forma del conjunto de datos: (5110, 12)

Información del conjunto de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [5]:
print("\nEstadísticas descriptivas:")
print(df.describe())

print("\nValores faltantes:")
print(df.isnull().sum())

print("\nDistribución de la variable objetivo:")
print(df['stroke'].value_counts())
print(df['stroke'].value_counts(normalize=True))


Estadísticas descriptivas:
                 id          age  hypertension  heart_disease  \
count   5110.000000  5110.000000   5110.000000    5110.000000   
mean   36517.829354    43.226614      0.097456       0.054012   
std    21161.721625    22.612647      0.296607       0.226063   
min       67.000000     0.080000      0.000000       0.000000   
25%    17741.250000    25.000000      0.000000       0.000000   
50%    36932.000000    45.000000      0.000000       0.000000   
75%    54682.000000    61.000000      0.000000       0.000000   
max    72940.000000    82.000000      1.000000       1.000000   

       avg_glucose_level          bmi       stroke  
count        5110.000000  4909.000000  5110.000000  
mean          106.147677    28.893237     0.048728  
std            45.283560     7.854067     0.215320  
min            55.120000    10.300000     0.000000  
25%            77.245000    23.500000     0.000000  
50%            91.885000    28.100000     0.000000  
75%           1

In [6]:
# Preprocesamiento de datos
# Eliminar columna ID ya que no es útil para la predicción
df = df.drop('id', axis=1)

In [7]:
# Manejar valores faltantes en IMC
df['bmi'] = pd.to_numeric(df['bmi'], errors='coerce')  # Convertir a numérico, establecer errores en NaN


In [10]:
# Dividir en características y objetivo
X = df.drop('stroke', axis=1)
y = df['stroke']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Definir pipelines de preprocesamiento
# Identificar tipos de columnas
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

In [12]:
# Crear pipelines de preprocesamiento
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [13]:
# Combinar todos los pasos de preprocesamiento
preprocessor = ColumnTransformer([
    ('categorical', categorical_pipeline, categorical_cols),
    ('numerical', numerical_pipeline, numerical_cols)
])

In [15]:
# Crear y evaluar modelo de referencia
baseline_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DummyClassifier(strategy='most_frequent'))
])
baseline_results = cross_validate(
    baseline_model, 
    X_train, 
    y_train, 
    cv=5, 
    return_train_score=True,
    scoring=['accuracy', 'precision', 'recall', 'f1']
)

print("\nResultados del Modelo de Referencia:")
for metric in ['accuracy', 'precision', 'recall', 'f1']:
    train_scores = baseline_results[f'train_{metric}']
    test_scores = baseline_results[f'test_{metric}']
    print(f"{metric}:")
    print(f"  Entrenamiento - Media: {np.mean(train_scores):.4f}, Desv. Estándar: {np.std(train_scores):.4f}")
    print(f"  Validación - Media: {np.mean(test_scores):.4f}, Desv. Estándar: {np.std(test_scores):.4f}")


C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\User


Resultados del Modelo de Referencia:
accuracy:
  Entrenamiento - Media: 0.9543, Desv. Estándar: 0.0001
  Validación - Media: 0.9543, Desv. Estándar: 0.0006
precision:
  Entrenamiento - Media: 0.0000, Desv. Estándar: 0.0000
  Validación - Media: 0.0000, Desv. Estándar: 0.0000
recall:
  Entrenamiento - Media: 0.0000, Desv. Estándar: 0.0000
  Validación - Media: 0.0000, Desv. Estándar: 0.0000
f1:
  Entrenamiento - Media: 0.0000, Desv. Estándar: 0.0000
  Validación - Media: 0.0000, Desv. Estándar: 0.0000


C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
# Crear y evaluar modelo principal
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, max_depth=5))
])

model_results = cross_validate(
    model, 
    X_train, 
    y_train, 
    cv=5, 
    return_train_score=True,
    scoring=['accuracy', 'precision', 'recall', 'f1']
)

print("\nResultados del Modelo Random Forest:")
for metric in ['accuracy', 'precision', 'recall', 'f1']:
    train_scores = model_results[f'train_{metric}']
    test_scores = model_results[f'test_{metric}']
    print(f"{metric}:")
    print(f"  Entrenamiento - Media: {np.mean(train_scores):.4f}, Desv. Estándar: {np.std(train_scores):.4f}")
    print(f"  Validación - Media: {np.mean(test_scores):.4f}, Desv. Estándar: {np.std(test_scores):.4f}")


C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\User


Resultados del Modelo Random Forest:
accuracy:
  Entrenamiento - Media: 0.9543, Desv. Estándar: 0.0001
  Validación - Media: 0.9543, Desv. Estándar: 0.0006
precision:
  Entrenamiento - Media: 0.0000, Desv. Estándar: 0.0000
  Validación - Media: 0.0000, Desv. Estándar: 0.0000
recall:
  Entrenamiento - Media: 0.0000, Desv. Estándar: 0.0000
  Validación - Media: 0.0000, Desv. Estándar: 0.0000
f1:
  Entrenamiento - Media: 0.0000, Desv. Estándar: 0.0000
  Validación - Media: 0.0000, Desv. Estándar: 0.0000


C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
# Crear y evaluar modelo con SMOTE (para desequilibrio de clases)
imb_model = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42, max_depth=5))
])

imb_model_results = cross_validate(
    imb_model, 
    X_train, 
    y_train, 
    cv=5, 
    return_train_score=True,
    scoring=['accuracy', 'precision', 'recall', 'f1']
)

print("\nResultados del Modelo Random Forest con SMOTE:")
for metric in ['accuracy', 'precision', 'recall', 'f1']:
    train_scores = imb_model_results[f'train_{metric}']
    test_scores = imb_model_results[f'test_{metric}']
    print(f"{metric}:")
    print(f"  Entrenamiento - Media: {np.mean(train_scores):.4f}, Desv. Estándar: {np.std(train_scores):.4f}")
    print(f"  Validación - Media: {np.mean(test_scores):.4f}, Desv. Estándar: {np.std(test_scores):.4f}")



Resultados del Modelo Random Forest con SMOTE:
accuracy:
  Entrenamiento - Media: 0.7683, Desv. Estándar: 0.0048
  Validación - Media: 0.7537, Desv. Estándar: 0.0185
precision:
  Entrenamiento - Media: 0.1433, Desv. Estándar: 0.0027
  Validación - Media: 0.1215, Desv. Estándar: 0.0073
recall:
  Entrenamiento - Media: 0.8168, Desv. Estándar: 0.0222
  Validación - Media: 0.7006, Desv. Estándar: 0.0388
f1:
  Entrenamiento - Media: 0.2439, Desv. Estándar: 0.0046
  Validación - Media: 0.2069, Desv. Estándar: 0.0113


In [18]:
# Entrenamiento y evaluación del modelo final
final_model = imb_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

print("\nEvaluación del Modelo Final en Conjunto de Prueba:")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred))



Evaluación del Modelo Final en Conjunto de Prueba:

Matriz de Confusión:
[[719 241]
 [ 12  50]]

Informe de Clasificación:
              precision    recall  f1-score   support

           0       0.98      0.75      0.85       960
           1       0.17      0.81      0.28        62

    accuracy                           0.75      1022
   macro avg       0.58      0.78      0.57      1022
weighted avg       0.93      0.75      0.82      1022



C:\Users\Diego\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [19]:
# Importancia de características
if hasattr(final_model['classifier'], 'feature_importances_'):
    # Obtener nombres de características después del preprocesamiento
    ohe = final_model['preprocessor'].named_transformers_['categorical']['onehot']
    cat_features = ohe.get_feature_names_out(categorical_cols).tolist()
    all_features = cat_features + numerical_cols
    
    # Obtener importancias de características
    importances = final_model['classifier'].feature_importances_
    
    # Crear un DataFrame para mejor visualización
    feature_importance = pd.DataFrame({
        'Característica': all_features,
        'Importancia': importances
    }).sort_values(by='Importancia', ascending=False)
    
    print("\nImportancia de Características:")
    print(feature_importance.head(10))  # Mostrar las 10 características principales


Importancia de Características:
                    Característica  Importancia
10                             age     0.421451
1                 ever_married_Yes     0.152472
14                             bmi     0.071436
13               avg_glucose_level     0.071223
6             Residence_type_Urban     0.044061
7   smoking_status_formerly smoked     0.036668
5               work_type_children     0.032028
3                work_type_Private     0.029821
12                   heart_disease     0.028982
0                      gender_Male     0.027042
